In [139]:
import re
import pandas as pd
from bokeh.plotting import figure, show, output_file
from bokeh.models import Select, ColumnDataSource, CustomJS
from bokeh.layouts import column
from bokeh.models import Span
from bokeh.palettes import Category10
from bokeh.models import HoverTool

import matplotlib.pyplot as plt
import datetime as dt

In [140]:
fred_input_folder = '/zine/HPC02S1/ex-dveloza/AGORA/apps/fred_colombia_implementation/fred_input_files'
synthetic_pop_data_folder = '/zine/HPC02S1/ex-dveloza/AGORA/apps/synthetic_populations/data'

In [141]:
df_mun_info = pd.read_csv(f'{synthetic_pop_data_folder}/processed_data/geodata/geoinfo_municipios_colombia.csv')[['COD_DEPTO','NOM_DEPART']].drop_duplicates()
dept_cods = df_mun_info.COD_DEPTO.sort_values()[:-1]

In [142]:
dataframes_mov = {}
dataframes_shelter = {}
for dept_code in dept_cods:
    mobility_file_path  = f'{fred_input_folder}/mobility_trends/{dept_code}_mobility_trends.csv'
    shelter_file_path   = f'{fred_input_folder}/shelter_trends/{dept_code}_shelter_trends.csv'

    df_mobility = pd.read_csv(mobility_file_path)
    df_shelter  = pd.read_csv(shelter_file_path)

    df_mobility['date'] = pd.to_datetime(df_mobility['date'])
    df_shelter['date']  = pd.to_datetime(df_shelter['date'])

    df_mobility['mobility_trend'] = 1 + df_mobility['mobility_trend']

    dept_name = df_mun_info[df_mun_info.COD_DEPTO == dept_code].NOM_DEPART.tolist()[0]

    dataframes_mov[dept_name]       = df_mobility
    dataframes_shelter[dept_name]   = df_shelter

In [143]:
# Convert DataFrames to ColumnDataSource for mobility and shelter
sources_mov = {name: ColumnDataSource(df) for name, df in dataframes_mov.items()}
sources_shelter = {name: ColumnDataSource(df) for name, df in dataframes_shelter.items()}

# Initial ColumnDataSource for mobility and shelter
initial_source_mov = dataframes_mov['CUNDINAMARCA']
initial_source_shelter = dataframes_shelter['CUNDINAMARCA']

# Create the figure with datetime x-axis
p = figure(x_axis_type='datetime', title="Mobility and Shelter Trends Over Time", width=800, height=400)

# Add lines for mobility and shelter
mobility_line = p.line(x='date', y='mobility_trend', source=initial_source_mov, line_width=2, color='blue', legend_label='Mobility Trend')
shelter_line = p.line(x='date', y='shelter_trend', source=initial_source_shelter, line_width=2, color='green', legend_label='Shelter Trend')

# Add hover tool
p.add_tools(HoverTool(tooltips=[('Date', '@date{%F}'), ('Trend', '@shelter_trend')], formatters={'@date': 'datetime'}))

# Adding a horizontal line at y = 1
hline = Span(location=1, dimension='width', line_color='red', line_width=3)
p.add_layout(hline)

# Set legend click policy
p.legend.click_policy = "hide"

# y axis
p.y_range.start = 0.05
p.y_range.end = 1.55

# Select widget with JavaScript callback for changing data source
select = Select(title="Choose Department:", value='CUNDINAMARCA', options=list(sources_mov.keys()))

select.js_on_change('value', CustomJS(args=dict(sources_mov=sources_mov, sources_shelter=sources_shelter, mobility_line=mobility_line, shelter_line=shelter_line), code="""
    const selected = cb_obj.value;
    const new_source_mov = sources_mov[selected];
    const new_source_shelter = sources_shelter[selected];

    // Update data source for mobility and shelter lines
    mobility_line.data_source.data = new_source_mov.data;
    shelter_line.data_source.data = new_source_shelter.data;
    
    // Emit the change to update the plot
    mobility_line.data_source.change.emit();
    shelter_line.data_source.change.emit();
"""))

# Layout
layout = column(select, p)

# Output to HTML
output_file('../output_files/mobility_shelter_trends.html')
show(layout)